# Mapeo de Hénon

- Hénon propuso, en 1976, un mapeo en 2D sencillo que exhibiera las propiedades esenciales del sistema de ecuaciones de primer orden acopladas propuestas por Lorentz, con el objetivo de poder estudiar de forma sencilla dichas propiedades.

- El mapeo imita una sección de Poincaré del atractor de Lorentz.

- El mapeo propuesto es el siguiente 

\begin{equation}
\begin{split}
    T:\,&x_{i+1}=y_{i+1}-ax_{i}^{2}\\
    &y_{i+1}=bx_{i}
\end{split}
\end{equation}





- Dicho mapeo se puede descomponer en tres etapas, i.e., $T\,=\,T'''T''T$. Si se toma como condición inicial una elipse:


![](doblamiento_T1.png)

- La primera etapa consiste en un doblamiento:

\begin{equation}
\begin{split}
    T':\,&x'=x\\
    &y'=y+1-ax^{2}
\end{split}
\end{equation}

![](doblamiento_T2.png)

- La segunda etapa consiste en una contracción en $x$:

\begin{equation}
\begin{split}
    T'':\,&x''=bx'\\
    &y''=y
\end{split}
\end{equation}


![](doblamiento_T3.png)

- La tercera etapa consiste en un reorientación en el eje x:

\begin{equation}
\begin{split}
    T''':\,&x'''=y''\\
    &y'''=x''
\end{split}
\end{equation}

![](doblamiento_T4.png)

- Es un mapeo con Jacobiano constante:

\begin{equation}
    \frac{\partial (x_{i+1},y_{i+1})}{\partial (x_{i},y_{i})}=-b
\end{equation}



- Tiene dos puntos invariantes

\begin{equation}
\begin{split}
    &x=\frac{-(1-b)\pm\sqrt{(1-b)^{2}+4a}}{2a}\\
    &y=bx
\end{split}
\end{equation}



- Dichos puntos son reales para 
\begin{equation}
a>a_{0}=-\frac{1}{4}(1-b)^{2}.
\end{equation}


En este caso, un punto es siempre linealmente inestable, mientras que el otro es inestable para 

\begin{equation}
a>a_{1}=\frac{3}{4}(1-b)^{2}.
\end{equation}

# Áreas

## Mapeos en 2D
### Estabilidad de puntos fijos

Un mapeo en dos dimensiones se puede caracterizar, al igual que un mapeo en una dimensión, con la estabilidad de sus puntos fijos, sus órbitas periódicas y otras propiedades que dependen de los parámetros del mapeo. 

Sea $f(\vec{x},\alpha)$ un mapeo en dos dimensiones, con $ \vec{x}=(x,y)$  y $\alpha$ un conjunto de parámetros de los que depende el mapeo.

Los puntos fijos serán aquellos que satisfagan

$$f(\vec{x}^*,\alpha)=\vec{x}^*$$

Para mapeos lineales de la forma 

$$f(\vec{x})=A\vec{x}$$  (con A una matriz de $2\times2$) se tiene que sus puntos fijos son los que satisfacen 

$$A\vec{x}^*=\vec{x}^*$$

ó bien 

$$ (A-I)\vec{x}^*=0$$

Si $(A-I)$ es no singular, entonces $\vec{x}^*$ es el unico punto de equilibrio del sistema, pero si $(A-I)$ es singular, hay una familia de puntos de equlibrio. En tal caso se define

$$\vec{y}=\vec{x}-\vec{x}^*$$ 

y se obtiene un nuevo mapeo 

$$f_y(\vec{y})=A\vec{y}$$ 
cuyos puntos de equlibrio satisfacen

$$\vec{y}^*=A\vec{y}^*$$

Así las ropiedades de estabilidad de un punto de equilibrio $\vec{x}^*\neq 0 $son las mismas que las de un punto de equilibrio $\vec{x}=0$. Sea

$$J=P^{-1}AP$$

la forma canónica de Jordan de A. Sea también 

$$\vec{y}=P^{-1}\vec{x}$$

Entonces el sistema original se transforma a 

$$f_y(\vec{y})=J\vec{y}$$ 


Hay tre posibles formas canónicas de Jordan que se pueden tener, como vimos en clase. Estas formas son:
\begin{equation*}
\left( 
\begin{array}{cc}
\alpha & -\beta \\
\beta & \alpha 
\end{array} 
\right), 
\left( 
\begin{array}{cc}
\lambda & 1 \\
0 & \lambda 
\end{array} 
\right),
\left( 
\begin{array}{cc}
\lambda & 0 \\
0 & \mu
\end{array} 
\right),
\end{equation*}
donde $\alpha, \beta, \lambda$ y $\mu$ son reales, con $\beta\ne 0$ y $\mu\ne\lambda$. En esta forma, los elementos de la diagonal son los eigenvalores, excepto para la primer matriz, cuyos eigenvalores son $\alpha\pm i \beta$.

Para el caso general en que el mapeo $f(\vec{x},\alpha)$ no es lineal, se puede linealizar con la matriz jacbiana $J_b$ y así estudiar localmente el comportamiento alrededor de los puntos fijos del mapeo.

Por ahora estudiemos el caso en el que los eigenvalores son dos reales distintos. 

Cuando se tienen dos eigenvalores reales distintos, los puntos fijos pueden ser:

- Puntos fijos aasíntoticamente estables: $\lambda_1\lt\lambda_2\lt1$
- Puntos fijos  inestables: $\lambda_1\gt\lambda_2\gt1$
- Puntos fijos silla: $0 \lt \lambda_1 \lt 1, \lambda_2\gt1$
- Puntos fijos degenerados: $\lambda_1=1, \lambda_1 \gt \lambda_2$
- Punto fijo estable:$-1\lt\lambda_1\lt 0 \lt\lambda_2\lt1$
- Punto fijo estable: $-1\lt\lambda_1\lt\lambda_2\lt0$
- Punto fijo silla: $\lambda_1\lt -1 \lt \lambda_2 \lt 0$
- Punto fijo silla: $\lambda_1\lt -1$ y  $ 0\lt\lambda_2 \lt 1$
- Punto fijo silla: $-1\lt\lambda_1\lt 0$ y  $ 1\lt\lambda_2$

Nota: Los puntos fijos silla son inestables.

Esto lo usamos  para estudiar los puntos fijos del  **mapeo clásico de Hénon** que  está definido por 

$$ \begin{array}{cc}
x_{n+1}=& 1-ax^2_n+y_n \\
y_{n+1}= & bx_n 
\end{array} $$


La matriz jacobiana de este mapeo es:

\begin{equation*}
J_b=\left( 
\begin{array}{cc}
-2ax & 1 \\
b & 0 
\end{array} 
\right)
\end{equation*}

Sus eigenvalores están dados por 

$$\lambda_{\pm}=-ax\pm\sqrt{(ax)^2+b}$$

Estos eigenvalores son reales y diferentes simpre que se satisfaga $b\gt-(ax)^2$

Los puntos fijos del mapeo de Hénon están dados por 

$$x=\frac{(1-b)}{2a}\pm 1/2\sqrt{(1-b)^2+4a}$$

$$y=bx$$

Por lo que existen dos puntos fijos en el plano $XY$ para cuando 

$$a\gt-1/4(1-b)^2$$

y un solo punto fijo en el plano $XY$ si
$$a=-1/4(1-b)^2$$

Escribamos algunas funciones que nos permitirán estudiar la estabilidad de los puntos fijos del mapeo de Hénon.

In [1]:
"""Función que define el mapeo de Henon en 2D"""
function Henon_map(x,y,a,b)
    fx=y+1-a*x^2
    fy=b*x
    return fx,fy
end 

Henon_map

In [2]:
function Fixed_points(a,b)
    xf1=-(1-b)/(2a)+(1/2a)*sqrt((1-b)^2+4a)
    xf2=-(1-b)/(2a)-(1/2a)*sqrt((1-b)^2+4a)
    yf1=b*xf1
    yf2=b*xf2
    return (xf1,yf1),(xf2,yf2)
end 

Fixed_points (generic function with 1 method)

In [3]:
function Jacobian_henon(x,y,a,b)
    Df=[[-2a*x,b] [1,0]]
    return Df
end 

Jacobian_henon (generic function with 1 method)

In [4]:
function EigenVals_JacHen(x,y,a,b)
    λ₁=-a*x+sqrt((a*x)^2+b)
    λ₂=-a*x-sqrt((a*x)^2+b)
    return λ₁,λ₂
end 

EigenVals_JacHen (generic function with 1 method)

## Conjutos de Julia

Ya que sabemos cómo determinar la estabilidad de los puntos fijos del mapeo de Hénon, pero necesitamos estudiar las órbitas que se obtienen al iterar $N$ veces en el mapeo de Hénon una condición inicial $\vec{x}_0$. 

Tal como hicimos en la tarea 2, exploremos una región grande el el plano $XY$ para darnos una idea de cuáles son las condiciones iniciales que al iterarse convergen a alguna órbita acotada en el espacio $XY$ y cuáles otras condiciones "explotan" rápidamente después de iterar el mapeo una cantidad finita de veces.

La siguiente función es un algoritmo que construye el conjunto de Julia para una región en el espacio $XY$

In [6]:
"""Función que grafica el conjunto de Julia generado por el mapeo f(z,c), en donde z 
es la variable en el plano complejo y c el parametro que caracteriza al 
sistema dinamico z_{n+1}=f(z_n)

init_cond =  la malla de condiciones iniciales en el plano complejo 

m =  numero limite de iteraciones del mapeo

R= radio del circulo en el plano complejo que se toma como criterio de divergencia

"""
function Julia_set_Henon(a,b,xrange,yrange,maxiter,R)
    
    mx=length(xrange)
    my=length(yrange)
    iterandos=zeros(my,mx) 
    for i in 1:mx
        for j in 1:my
            x=xrange[i]
            y=yrange[my+1-j]
                for k in 1:maxiter
                x,y=Henon_map(x,y,a,b)
                iterandos[j,i]=k
                    if sqrt(x^2+y^2)>=R 
                    break
                    end 
                end
        end 
    end
    return iterandos
end

Julia_set_Henon

# Caso conservativo 

En este caso $b=-1$ y $b=1$. Para que existan puntos fijos reales se debe cumplir $a>a_{0}=-\frac{1}{4}(1-b)^{2}$.

Para $b=-1$, $a=1.4$ se tiene lo siguiente:

In [7]:
a=1.4
b=-1
xf1,xf2=Fixed_points(a,b)

((0.39228095605926205,-0.39228095605926205),(-1.8208523846306908,1.8208523846306908))

In [8]:
Jac1=Jacobian_henon(xf1[1],xf1[2],a,b)
Jac2=Jacobian_henon(xf2[1],xf2[2],a,b);

In [9]:
λ1,λ2=eigvals(Jac1)

2-element Array{Complex{Float64},1}:
 -0.549193+0.835695im
 -0.549193-0.835695im

In [10]:
λ3,λ4=eigvals(Jac2)

2-element Array{Float64,1}:
 4.89406 
 0.204329

Aquí el primer punto fijo $\vec{x_1^*}=(0.3922...,-0.3922...)$ nos da una matriz Jacobiana cuyos valores propios son complejos. Notemos que la norma de estos valores propios es 

In [11]:
abs(λ1)

1.0

Este caso corresponde a puntos fijos que se denominan **centros estables**, y dan lugar a órbitas periódicas cerradas en el espacio fase como mostraremos más adelante. 

El segundo punto fijo correspondiente a $\vec{x_2^*}=(-1.82085...,1.82085...)$ coresponde a un punto silla inestable ya que ambos eigenvalores son reales y satisfacen un de los criterios arriba mencionados.

Por tanto el punto fijo en el caso **conservativo**, el punto fijo

$$\vec{x}_1^*=(0.84515...,0.84515...)$$ 

también corresponde  a un punto silla (inestable).


In [12]:
λ1,λ2=EigenVals_JacHen(xf2[1],xf2[2],a,b)

(4.894057231540714,0.2043294454252198)

Por tanto el punto fijo en el caso **conservativo**, el punto fijo

$$\vec{x}_2^*=(-0.84515...,-0.84515...)$$ 

también corresponde  a un punto silla (inestable).


Este tipo de análisis realizado para los conjuntos de parámetros $a=1.4,b=0.3,-1$ se puede realizar para cuales quiera dos pares de parámetros (a,b) siempre y cuando se satisfaga la condición de que existan puntos fijos en el plano complejo. Esta condición como se menciono anteriormente es que hay dos puntos fijos en el plano $XY$ para cuando 

$$a\gt-1/4(1-b)^2$$

y un solo punto fijo en el plano $XY$ si
$$a=-1/4(1-b)^2$$. 


- A continuación se presenta el conjunto de Julia para $b=1.0$ y se varía $a$ de $-1$ a $3$.

![](JS_b_p1.gif)

Se puede observar que para que existan puntos fijos reales se debe cumplir $a>0$

- A continuación se presenta el conjunto de Julia para $b=-1.0$ y se varía $a$ de $-2$ a $4$.

![](JS_b_m1.gif)

Se puede observar que para que existan puntos fijos reales se debe cumplir $a>-1$

# Caso disipativo

En este caso $b\neq|1|$. Se escogen los parámetros $a=1.4,b=0.3$

In [13]:
a=1.4
b=0.3
xf1,xf2=Fixed_points(a,b);

In [14]:
xf1

(0.6313544770895047,0.1894063431268514)

In [15]:
xf2

(-1.1313544770895048,-0.3394063431268514)

In [16]:
λ1,λ2=EigenVals_JacHen(xf1[1],xf1[2],a,b)

(0.15594632230279393,-1.9237388581534067)

Por tanto vemos que el punto fijo 

$$\vec{x^*_1}=(0.63135...,0.18940...)$$ 

es un punto fijo silla, que son puntos inestables.

Para el segundo punto fijo se tiene que los eigenvalores estan dados por:

In [17]:
λ3,λ4=EigenVals_JacHen(xf2[1],xf2[2],a,b)

(3.259822097891452,-0.09202956204083912)

Por tanto vemos que eel segundo punto fijo correspondiente a 

$$\vec{x^*_2}=(-1.13135...,-0.33940...)$$

también corresponde a un punto fijo silla (inestable).

A continuación se presenta el conjunto de Julia para $b=0.3$ y se varía $a$ de $-0.4$ a $3$.

![](JS_b_03.gif)